In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

In [13]:
image_path = "/content/drive/MyDrive/Assignment/assignment/images"
floor = "/content/drive/MyDrive/Assignment/assignment/floor.png"
wall = "/content/drive/MyDrive/Assignment/assignment/wall.png"
car_shadow_mask = "/content/drive/MyDrive/Assignment/assignment/shadow_masks"
car_masks = "/content/drive/MyDrive/Assignment/assignment/car_masks"

In [14]:
def read_images_and_masks(index):
    car_image = cv2.imread(os.path.join(image_path, f"{index}.jpeg"))
    car_mask = cv2.imread(os.path.join(car_masks, f"{index}.png"), cv2.IMREAD_GRAYSCALE)
    shadow_mask = cv2.imread(os.path.join(car_shadow_mask, f"{index}.png"), cv2.IMREAD_GRAYSCALE)

    def read_and_crop(image_path):
      image = cv2.imread(str(image_path))
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      _, mask = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)
      # Find bounding box of non-white regions
      coords = cv2.findNonZero(255 - mask)  # Invert mask to find content
      if coords is not None:
          x, y, w, h = cv2.boundingRect(coords)
          return image[y:y + h, x:x + w]
      return image  # Return original if no cropping needed
    wall_image = read_and_crop(wall)
    floor_image = read_and_crop(floor)
    return car_image, car_mask, shadow_mask, wall_image, floor_image

In [15]:
def preprocess_masks(mask):
    kernel = np.ones((5, 5), np.uint8)
    mask_cleaned = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask_cleaned = cv2.morphologyEx(mask_cleaned, cv2.MORPH_OPEN, kernel)
    return mask_cleaned

In [16]:
def adjust_car_size_and_position(car_image, car_mask, target_size, target_position, canvas_size):
    """
    Adjusts car size and positions it on the canvas with boundary checks.

    Parameters:
        car_image (numpy.ndarray): Original car image.
        car_mask (numpy.ndarray): Mask corresponding to the car image.
        target_size (tuple): Target size (width, height) for the resized car.
        target_position (tuple): Target position (x, y) for the car's top-left corner.
        canvas_size (tuple): Canvas dimensions (height, width).

    Returns:
        canvas (numpy.ndarray): Final canvas with adjusted car image.
    """
    # Resize car image and mask to the target size
    target_width, target_height = target_size
    resized_car = cv2.resize(car_image, (target_width, target_height), interpolation=cv2.INTER_AREA)
    resized_mask = cv2.resize(car_mask, (target_width, target_height), interpolation=cv2.INTER_AREA)

    # Create a blank canvas with the same dimensions as the canvas size
    canvas_height, canvas_width = canvas_size
    canvas = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)

    # Calculate the region on the canvas
    start_x, start_y = target_position
    end_x = min(start_x + target_width, canvas_width)
    end_y = min(start_y + target_height, canvas_height)

    # Calculate the valid region within the resized car
    region_width = end_x - start_x
    region_height = end_y - start_y
    if region_width > 0 and region_height > 0:
        resized_car = resized_car[:region_height, :region_width]
        resized_mask = resized_mask[:region_height, :region_width]

        # Safely place the resized car onto the canvas
        canvas[start_y:end_y, start_x:end_x] = resized_car

    return canvas


In [17]:
def create_background(wall, floor, output_size):
    wall_height = int(output_size[0] * 0.65)
    floor_height = output_size[0] - wall_height + 10
    wall_resized = cv2.resize(wall, (output_size[1], wall_height), interpolation=cv2.INTER_AREA)
    floor_resized = cv2.resize(floor, (output_size[1], floor_height), interpolation=cv2.INTER_AREA)
    background = np.vstack((wall_resized, floor_resized))
    return background

def replace_background(car_image, mask_image, background_image):
    # Resize the mask and background to match the car image
    mask_resized = cv2.resize(mask_image, (car_image.shape[1], car_image.shape[0]))
    background_resized = cv2.resize(background_image, (car_image.shape[1], car_image.shape[0]))

    # Convert the background to RGB if it is in BGR format
    background_resized = cv2.cvtColor(background_resized, cv2.COLOR_BGR2RGB)

    # Create a masked version of the car image
    masked_car = np.copy(car_image)
    masked_car[mask_resized == 0] = [0, 0, 0]  # Set pixels outside the mask to black

    # Create a masked version of the background
    masked_background = np.copy(background_resized)
    masked_background[mask_resized > 0] = [0, 0, 0]  # Set pixels inside the mask to black

    # Combine the car and background
    combined_image = masked_car + masked_background

    return combined_image

In [18]:
def add_shadows(image, shadow_mask, car_mask):
    shadow_mask = cv2.GaussianBlur(shadow_mask, (15, 15), 0)
    shadow_mask = cv2.resize(shadow_mask, (image.shape[1], image.shape[0]))
    shadow_mask = cv2.bitwise_and(shadow_mask, car_mask)  # Align shadow with car
    shadow = cv2.merge([shadow_mask] * 3)
    shadow_overlay = cv2.addWeighted(image, 1, shadow, -0.6, 0)
    return shadow_overlay
# def add_shadows_with_alignment(image, shadow_mask, car_mask):
#     # Resize and blur the shadow mask
#     shadow_mask = cv2.GaussianBlur(shadow_mask, (15, 15), 0)
#     shadow_mask = cv2.resize(shadow_mask, (image.shape[1], image.shape[0]))

#     # Find the bottommost white pixel in the car mask
#     rows, cols = np.where(car_mask == 255)
#     if len(rows) > 0:
#         bottom_row = max(rows)
#         shadow_mask_aligned = np.zeros_like(shadow_mask)
#         shadow_mask_aligned[bottom_row:] = shadow_mask[bottom_row:]
#     else:
#         shadow_mask_aligned = shadow_mask

#     # Align shadow to the car mask region
#     shadow_mask_final = cv2.bitwise_and(shadow_mask_aligned, car_mask)
#     shadow = cv2.merge([shadow_mask_final] * 3)

#     # Apply the shadow overlay
#     shadow_overlay = cv2.addWeighted(image, 1, shadow, -0.3, 0)
#     return shadow_overlay


In [19]:
def process_images():
    for i in range(1, 7):
        car, car_mask, shadow_mask, wall, floor = read_images_and_masks(i)

        # Resize and reposition cars
        # car, car_mask = resize_and_position_cars(car, car_mask)

        car_mask = preprocess_masks(car_mask)
        shadow_mask = preprocess_masks(shadow_mask)

        background = create_background(wall, floor, car.shape[:2])
        car_with_bg = replace_background(car, car_mask, background)
        # final_image = add_shadows(car_with_bg, shadow_mask, car_mask)
        result = add_shadows(car_with_bg, shadow_mask, car_mask)

        output_path = os.path.join("/content/drive/MyDrive/Assignment/assignment", f"output_{i}.jpg")
        cv2.imwrite(output_path, result)
        print(f"Processed and saved: {output_path}")

In [20]:
process_images()

Processed and saved: /content/drive/MyDrive/Assignment/assignment/output_1.jpg
Processed and saved: /content/drive/MyDrive/Assignment/assignment/output_2.jpg
Processed and saved: /content/drive/MyDrive/Assignment/assignment/output_3.jpg
Processed and saved: /content/drive/MyDrive/Assignment/assignment/output_4.jpg
Processed and saved: /content/drive/MyDrive/Assignment/assignment/output_5.jpg
Processed and saved: /content/drive/MyDrive/Assignment/assignment/output_6.jpg
